**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**

---


# Introduction

[Stack Overflow](https://stackoverflow.com/) is a widely beloved question and answer site for technical questions. You'll probably use it yourself as you keep using SQL (or any programming language). 

Their data is publicly available. What cool things do you think it would be useful for?

Here's one idea:
You could set up a service that identifies the Stack Overflow users who have demonstrated expertise with a specific technology by answering related questions about it, so someone could hire those experts for in-depth help.

In this exercise, you'll write the SQL queries that might serve as the foundation for this type of service.

As usual, run the following cell to set up our feedback system before moving on.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex6 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


Run the next cell to fetch the `stackoverflow` dataset.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

### 1) Explore the data

Before writing queries or **JOIN** clauses, you'll want to see what tables are available. 

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [9]:
# Get a list of available tables 
tables = list(client.list_tables(dataset))
list_of_tables = [table.table_id for table in tables] # Your code here

# Print your answer
print(list_of_tables)

# Check your answer
q_1.check()

['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 2) Review relevant tables

If you are interested in people who answer questions on a given topic, the `posts_answers` table is a natural place to look. Run the following cell, and look at the output.

In [10]:
# Construct a reference to the "posts_answers" table
answers_table_ref = dataset_ref.table("posts_answers")

# API request - fetch the table
answers_table = client.get_table(answers_table_ref)

# Preview the first five lines of the "posts_answers" table
client.list_rows(answers_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,59891494,None,<p>Method - 1</p>\n\n<p>It can be resolved by ...,None,None,0,None,2020-01-24 06:38:32.793000+00:00,None,2020-01-24 06:38:32.793000+00:00,None,None,None,None,8101535,59891493,2,0,None,None
1,59891498,None,<p>I hope this will help.</p>\n\n<pre><code>fu...,None,None,0,None,2020-01-24 06:38:38.377000+00:00,None,2020-01-24 06:38:38.377000+00:00,None,None,None,None,8427204,44183708,2,0,None,None
2,59891502,None,<p>Add class in row which you want to calculat...,None,None,0,None,2020-01-24 06:38:45.920000+00:00,None,2020-01-24 06:38:45.920000+00:00,None,None,None,None,6920236,59891186,2,0,None,None
3,59891504,None,"<pre><code>with open('file.txt','r') as f:\n ...",None,None,0,None,2020-01-24 06:38:50.550000+00:00,None,2020-01-24 06:38:50.550000+00:00,None,None,None,None,9851541,59884511,2,0,None,None
4,59891508,None,<p>I could fix the ALB ingress controller issu...,None,None,0,None,2020-01-24 06:39:06.210000+00:00,None,2020-01-24 06:39:06.210000+00:00,None,None,None,None,12599017,59627068,2,0,None,None


It isn't clear yet how to find users who answered questions on any given topic. But `posts_answers` has a `parent_id` column. If you are familiar with the Stack Overflow site, you might figure out that the `parent_id` is the question each post is answering.

Look at `posts_questions` using the cell below.

In [11]:
# Construct a reference to the "posts_questions" table
questions_table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
questions_table = client.get_table(questions_table_ref)

# Preview the first five lines of the "posts_questions" table
client.list_rows(questions_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,32550431,Errors while opening sample android studio pro...,<p>I am trying to open android studio project ...,NaN,0,0,None,2015-09-13 14:01:00.530000+00:00,1.0,2015-09-13 14:46:28.283000+00:00,2017-05-23 12:03:52.263000+00:00,None,-1.0,None,5325746,None,1,1,eclipse|android-ndk|apk,256
1,32561991,Is it ok to create databases in AWS RDS dynami...,<p>I am trying to create a new database so tha...,NaN,1,0,None,2015-09-14 10:00:39.170000+00:00,NaN,2015-09-14 10:31:03.397000+00:00,NaT,None,NaN,None,1429811,None,1,0,php|mysql|amazon-web-services|amazon-ec2|amazo...,256
2,32578769,Sparkle Cocoa: My Application is latest but it...,<p>I am using sparkle to update my application...,NaN,1,0,None,2015-09-15 06:03:39.060000+00:00,NaN,2017-11-30 13:18:29.040000+00:00,2015-09-15 08:55:12.927000+00:00,None,4221433.0,None,4221433,None,1,0,objective-c|cocoa|sparkle,256
3,32592875,rruleJS: Consideration of timezone for weekly ...,"<p>I'm using rruleJS (<a href=""https://github....",32601450.0,2,0,None,2015-09-15 18:11:50.917000+00:00,2.0,2017-03-03 22:29:23.250000+00:00,2015-09-16 09:22:56.343000+00:00,None,3593647.0,None,3593647,None,1,0,javascript|recurrence|rrule,256
4,32596387,How do I avoid shadowing an stdlib module in O...,<p>I have a program that uses the <code>Bytes<...,32596641.0,2,0,None,2015-09-15 21:58:51.343000+00:00,NaN,2015-09-16 08:34:48.730000+00:00,2015-09-16 08:34:48.730000+00:00,None,194635.0,None,90511,None,1,2,ocaml|ocaml-core,256


Are there any fields that identify what topic or technology each question is about? If so, how could you find the IDs of users who answered questions about a specific topic?

Think about it, and then check the solution by running the code in the next cell.

In [12]:
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
`posts_questions` has a column called `tags` which lists the topics/technologies each question is about.

`posts_answers` has a column called `parent_id` which identifies the ID of the question each answer is responding to.
`posts_answers` also has an `owner_user_id` column which specifies the ID of the user who answered the question.

You can join these two tables to:
- determine the `tags` for each answer, and then
- select the `owner_user_id` of the answers on the desired tag.

This is exactly what you will do over the next few questions.


### 3) Selecting the right questions

A lot of this data is text. 

We'll explore one last technique in this course which you can apply to this text.

A **WHERE** clause can limit your results to rows with certain text using the **LIKE** feature. For example, to select just the third row of the `pets` table from the tutorial, we could use the query in the picture below.

![](https://i.imgur.com/RccsXBr.png) 

You can also use `%` as a "wildcard" for any number of characters. So you can also get the third row with:

```
query = """
        SELECT * 
        FROM `bigquery-public-data.pet_records.pets` 
        WHERE Name LIKE '%ipl%'
        """
```

Try this yourself. Write a query that selects the `id`, `title` and `owner_user_id` columns from the `posts_questions` table. 
- Restrict the results to rows that contain the word "bigquery" in the `tags` column. 
- Include rows where there is other text in addition to the word "bigquery" (e.g., if a row has a tag "bigquery-sql", your results should include that too).

In [15]:
# Your code here
questions_query = """
                  SELECT id, title, owner_user_id,
                  FROM `bigquery-public-data.stackoverflow.posts_questions`
                  WHERE tags LIKE '%bigquery%'
                  """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
questions_query_job = client.query(questions_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
questions_results = questions_query_job.to_dataframe() # Your code goes here

# Preview results
print(questions_results.head())

# Check your answer
q_3.check()

         id                                              title  owner_user_id
0  29966463  How to extract unique days between two timesta...      4002069.0
1  30046367  bigquery row_number group by some rows based o...      2024056.0
2  30102965  Incorrect and inconsistent user counts using G...      2642218.0
3  29899097        BigQuery select data within a time interval      4213553.0
4  30201926       BigQuery streaming data from site javaScript      4051341.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 4) Your first join
Now that you have a query to select questions on any given topic (in this case, you chose "bigquery"), you can find the answers to those questions with a **JOIN**.  

Write a query that returns the `id`, `body` and `owner_user_id` columns from the `posts_answers` table for answers to "bigquery"-related questions. 
- You should have one row in your results for each answer to a question that has "bigquery" in the tags.  
- Remember you can get the tags for a question from the `tags` column in the `posts_questions` table.

Here's a reminder of what a **JOIN** looked like in the tutorial:
```
query = """
        SELECT p.Name AS Pet_Name, o.Name AS Owner_Name
        FROM `bigquery-public-data.pet_records.pets` as p
        INNER JOIN `bigquery-public-data.pet_records.owners` as o 
            ON p.ID = o.Pet_ID
        """
```

It may be useful to scroll up and review the first several rows of the `posts_answers` and `posts_questions` tables.  

In [22]:
# Your code here
answers_query = """
                SELECT a.id, a.body, a.owner_user_id
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q 
                INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                    ON q.id = a.parent_id
                WHERE q.tags LIKE '%bigquery%'
                """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
answers_query_job = client.query(answers_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
answers_results = answers_query_job.to_dataframe() # Your code goes here

# Preview results
print(answers_results.head())

# Check your answer
q_4.check()

         id                                               body  owner_user_id
0  59324764  <p>You might consider to use <a href="https://...      9928809.0
1  59337914  <p>In this example, I am parsing XML elements ...      5212614.0
2  59339167  <p>Given this example:</p>\n\n<pre><code>creat...      3270427.0
3  59349619  <p>Thanks to the comment by Blundering Philoso...     11152289.0
4  59355434  <p>You can install your own Python libraries i...     12232518.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 5) Answer the question
You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that has a single row for each user who answered at least one question with a tag that includes the string "bigquery". Your results should have two columns:
- `user_id` - contains the `owner_user_id` column from the `posts_answers` table
- `number_of_answers` - contains the number of answers the user has written to "bigquery"-related questions

In [25]:
# Your code here
bigquery_experts_query = """
                        SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
                        FROM `bigquery-public-data.stackoverflow.posts_questions` AS q 
                        INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                            ON q.id = a.parent_id
                        WHERE q.tags LIKE '%bigquery%'
                        GROUP BY a.owner_user_id
                        """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe() # Your code goes here

# Preview results
print(bigquery_experts_results.head())

# Check your answer
q_5.check()

     user_id  number_of_answers
0  5990514.0                  9
1  5071793.0                 50
2  4490873.0                267
3  1086647.0                  1
4    44385.0                  1


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 6) Building a more generally useful service

How could you convert what you've done to a general function a website could call on the backend to get experts on any topic?  

Think about it and then check the solution below.

In [ ]:
q_6.solution()

# Congratulations!

You know all the key components to use BigQuery and SQL effectively. Your SQL skills are sufficient to unlock many of the world's largest datasets.

Want to go play with your new powers?  Kaggle has BigQuery datasets available [here](https://www.kaggle.com/datasets?sortBy=hottest&group=public&page=1&pageSize=20&size=sizeAll&filetype=fileTypeBigQuery).

# Feedback

Bring any questions or feedback to the [Learn Discussion Forum](https://www.kaggle.com/learn-forum).

---
**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum) to chat with other Learners.*